Dataset yang digunakan ini menunjukkan persentase fat, protein, dan food yang dikonsumsi dari setiap jenis makanan yang terdaftar. Beberapa bagian akhir dataset juga mencakup persentase obesitas(Obesity), kurang gizi(Undernourished), dan kasus COVID-19 yang Terkonfirmasi(Confirmed)/Kematian(Deaths)/Sembuh(Recovered)/Aktif(Active).

(Catatan: Semua data memiliki unit '%' (persen) kecuali Populasi, unit persen tersebut dihitung berdasarkan jumlah populasi).

# Import Library yang dibutuhkan

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Import dataset yang dibutuhkan menggunakan pandas dan menyimpan dalam variabel yang mudah diingat

Dataset yang saya gunakan dari
https://www.kaggle.com/mariaren/covid19-healthy-diet-dataset/notebooks

In [ ]:
df_food = pd.read_csv('../input/covid19-healthy-diet-dataset/Food_Supply_Quantity_kg_Data.csv')
df_protein = pd.read_csv('../input/covid19-healthy-diet-dataset/Protein_Supply_Quantity_Data.csv')
df_fat = pd.read_csv('../input/covid19-healthy-diet-dataset/Fat_Supply_Quantity_Data.csv')
df_category = pd.read_csv('../input/covid19-healthy-diet-dataset/Supply_Food_Data_Descriptions.csv')

# Memeriksa 5 data teratas dari masing masing dataset

In [ ]:
df_food.head()

In [ ]:
df_protein.head()

In [ ]:
df_fat.head()

In [ ]:
df_category.head()

# Memisahkan kolom pada supply dataset antara kategori makanan dan bukan makanan

In [ ]:
# menempatkan semua data kedalam all_columns
all_columns = df_food.columns.tolist()
print(f'All Columns : {all_columns} \n\n')

# menempatkan data yang merupakan kategori food dalam supply_columns
supply_columns = df_category['Categories'].tolist()
print(f'Supply Columns : {supply_columns} \n\n')

# menempatkan data yang merupakan kategori non food dalam non_supply_columns
non_supply_columns = [i for i in all_columns if i not in supply_columns]
print(f'Non-Supply Columns : {non_supply_columns}')

# Menggabungkan DataFrame 
dengan menggabungkan data yang merupakan kategori non food/non supply dengan kategori supply dan data yang merupakan supply dari tiap df dibedakan dengan nama daf nya misalkan 'Meat_food', 'Meat_fat, dan 'Meat_protein'

In [ ]:
# menyimpan data yang merupakan non supply
df_non_supply_new = df_food[non_supply_columns]
# menyimpan data food yang merupakan supply dan ditambahkan kolom country 
df_food_new = df_food[['Country']+supply_columns]
# menyimpan data protein yang merupakan supply dan ditambahkan kolom country
df_protein_new = df_protein[['Country']+supply_columns]
# menyimpan data fat yang merupakan supply dan ditambahkan kolom country
df_fat_new = df_fat[['Country']+supply_columns]

In [ ]:
# Menggabungkan dataset yang sudah disimpan dengan variabel di atas dan menambahkan suffix berbeda untuk setiap supply dataset menggunakan pd.merge
df1=pd.merge(df_food_new, df_protein_new, on="Country", suffixes=('_food', '_protein'), how = 'outer')
# Menambahkan _fat 
df_fat_new_cols = [i + '_fat' if i != 'Country' else i for i in df_fat_new.columns]
df_fat_new.columns = df_fat_new_cols

df2=pd.merge(df_fat_new, df_non_supply_new, how='outer', on='Country')
df3=pd.merge(df1, df2, left_index=True, right_index=True, how='left', on='Country')
df3.columns.tolist()

In [ ]:
Menampilkan df hasil penggabungan data

In [ ]:
df3

# Sorting dan Filtering
Mengurutkan data (sort) berdasarkan kolom kematian ('Deaths') dari yang terbesar.

In [ ]:
# Urutkan (sorting) dan print hasil sorting
df_sort = df3.sort_values(by=['Deaths'], ascending=False, ignore_index=True)
df_sort

Menampilkan Negara Indonesia dalam urutan ke berapa

In [ ]:
indo = list(df_sort[df_sort['Country']== 'Indonesia'].index+1)[0]

print('Indonesia urutan : {}'.format(indo))

> Di dapatkan Negara Indonesia terdapat pada urutan ke-85 dari 170 negara menurut urutan persentase kematian, dan didapatkan dari hasil ini berarti Indonesia berada di 50% dari keseluruhan negara yang terdata dalam dataset berdasarkan tingkat kematian akibat COVID-19 setelah diurutkan dari yang terbesar.

Menghitung persentase jumlah kematian di Indonesia

In [ ]:
meninggal_persen = list(df_sort['Deaths'][df_sort['Country']=='Indonesia'])[0]
meninggal = list(df_sort['Deaths'][df_sort['Country']=='Indonesia']*df_sort['Population'][df_sort['Country']=='Indonesia']*0.01)[0]

print(f'Jumlah yang meninggal di Indonesia ada {round(meninggal_persen, 10)}% dari jumlah populasi.') # 10 digit
print(f'Jumlah yang meninggal di Indonesia ada {round(meninggal)} orang.')

# Binning, Grouping, dan Aggregating data

Melakukan binning pada df_fat menjadi 5 bagian disimpan pada kolom Confirmed Category dengan label Sangat Tidak Banyak', 'Tidak Banyak', 'Sedang', 'Cukup Banyak', 'Sangat Banyak'.

In [ ]:
# Melakukan binning pada kolom terkonfirmasi untuk DataFrame df_fat
labels = ['Sangat Tidak Banyak', 'Tidak Banyak', 'Sedang', 'Cukup Banyak', 'Sangat Banyak']

df_fat['Confirmed Category'] = pd.qcut((df_fat['Confirmed']), 5, labels=labels)

In [ ]:
# Melakukan grouping pada kolom hasil binning ('Confirmed Category'), kemudian aggregate menggunakan fungsi mean
cols = ['Obesity', 'Meat', 'Fruits - Excluding Wine', 'Fish, Seafood', 'Animal Products', 'Vegetal Products']

groupby_confirmed = df_fat.groupby('Confirmed Category')[cols].mean()
groupby_confirmed

In [ ]:
# Menyimpan DataFrame hasil binning, grouping, dan aggregating dalam bentuk csv

groupby_confirmed.to_csv('groupby_confirmed.csv')

# Bar Plot
Menggunakan data grouping pada soal sebelumnya untuk melakukan visualisasi Bar Plot

In [ ]:
# Gunakan DataFrame hasil grouping 
df = groupby_confirmed

# Lihat df 
display(df)

In [ ]:
# Normalisasi pada DataFrame df
df_norm = (df - df.min()) / (df.max() - df.min())

# Lihat df_norm dan pastikan data telah ternormalisasi
display(df_norm)

In [ ]:
# Buat bar plot dari df_norm 
x = np.arange(len(df_norm.columns))
y = df_norm.loc['Sangat Tidak Banyak']
y1 = df_norm.loc['Tidak Banyak']
y2 = df_norm.loc['Sedang']
y3 = df_norm.loc['Cukup Banyak']
y4 = df_norm.loc['Sangat Banyak']

In [ ]:
fig, ax = plt.subplots(figsize=(14,10))
width = 0.1
a1= ax.bar(x, y, width, label='Sangat Tidak Banyak')
a2= ax.bar(x+0.1, y1, width, label='Tidak Banyak')
a3= ax.bar(x+0.2, y2, width, label='Sedang')
a4= ax.bar(x+0.3, y3, width, label='Cukup Banyak')
a5= ax.bar(x+0.4, y4, width, label='Sangat Banyak')

#menambahkan label
def autolabel(rects):
    """Attach a text label above each bar in *rects*, displaying its height."""
    for rect in rects:
        height = rect.get_height()
        plt.text(rect.get_x() + rect.get_width() / 2, height, str(round(height,2)),
                    ha='center', va='bottom')
        
autolabel(a1)
autolabel(a2)
autolabel(a3)
autolabel(a4)
autolabel(a5)

plt.xticks(x + 0.2, ('Obesity', 'Meat', 'Fruits - Excluding Wine', 'Fish, Seafood', 'Animal Products','Vegetal Products'))
ax.set_title('Perbandingan Konsumsi Makanan Berdasarkan Tingkat Kasus COVID 19')
ax.legend(bbox_to_anchor=(0.95,1.1))
plt.show()

* dari visualisasi diatas menunjukkkan hasil obesity dari data confirmed pada df_fat menunjukkan hasil yang signifikan pada label 'Cukup Banyak' yang memungkinkan Pasien covid 19 banyak mengkonsumsi supply yang cenderung kurang sehat/menunjukkan tingkat obesitas yang cukup banyak
* dari visualisasi diatas menunjukkan konsumsi supply meat mendapatkan grafik yang signifikan pada label 'Sangat Banyak' jadi memungkinkan sebagian besar negara terkonfirmasi Codiv 19 ini mengkonsumsi supply meat yang banyak
* dari visualisasi diatas menunjukkan konsumsi Fruits - Exc. Wine dan Fish, Seafood mendapatkan grafik yang signifikan pada label yang sama yaitu 'Sangat Tidak Banyak' jadi kemungkinan negara-negara yang terdata dalam dataset ini masih kurang mengkonsumsi jenis supply Fruits - Exc. Wine dan Fish, Seafood
* dari visualisasi diatas menunjukkan konsumsi Animal Products medapatkan grafik yang signifikan pada label 'Sangat Banyak' jadi memungkinkan sebagian besar negara terkonfirmasi Codiv 19 ini mengkonsumsi supply Animal Products yang banyak
* dari visualisasi diatas menunjukkan konsumsi Vegetal Products medapatkan grafik yang signifikan pada label 'Tidak Banyak' jadi memungkinkan sebagian besar negara terkonfirmasi Codiv 19 ini kurang mengkonsumsi supply Vegetal Products

> Jadi, menurut visualisasi diatas negara-negara terkonfirmasi covid yang terdata dalam dataset ini menunjukkan tingkat obesitas yang cukup banyak dengan data supply makanan jenis daging-dagingan dan produk hewani yang menunjukkan konsumsi yang cenderung banyak sedangkan konsumsi suppy makanan jenis buah,ikan,seafood dan sayur-sayuran yang cenderung kurang.

# Seaborn Scatter plot

Visualisasi scatter plot (perpaduan seaborn dan matplotlib) dari df_food antara Supply Sayuran ('Vegetal Products') dan persentase terkonfirmasi ('Confirmed') COVID-19.

In [ ]:
fig, ax = plt.subplots(figsize=(18, 12))

V = df_food['Vegetal Products']
C = df_food['Confirmed']
A = df_food['Animal Products']
# mengalikan kolom death agar dapat terlihat dalam visualisasi
D = 200000*df_food['Deaths']

# menggunakan kolom persentase kematian ('Death') sebagai ukuran pointnya, produk hewani ('Animal Products') sebagai warna point dengan cmap 'plasma'.
map = plt.scatter(V,C, c=A, cmap='plasma', sizes=D)
sns.regplot(x="Vegetal Products", y="Confirmed", data=df_food, scatter=False, color='black');

# axis labeling
ax.set_xlabel('Vegetal Products')
ax.set_ylabel('Confirmed')
ax.set_title('Healthy Food and Covid-19')

fig.colorbar(map)
plt.show()

pada visualisasi diatas menunjukkan scatter plot jumlah konsumsi produk supply vegetal/sayuran dan kasus yang terkonfirmasi, dari visualisasi ini didapatkan semakin besar jumlah konsumsi vegetal products maka semakin sedikit juga confirmed case nya, point selanjutnya adalah penggunaan kolom death sebagai ukuran poinnya dimana sebakin besar ukuran poin dalam visualisasi scatter semakin besar juga persentasi kematiannya dan yang terakhir yaitu warna point yang menggunakan kolom produk hewani yang juga telah diberikan range perkiraannya dengan cmap